# **Market Segmentation**

---

# Latar Belakang  

Menggunakan dataset dari https://www.kaggle.com/vik2012kvs/high-value-customers-identification  
Toko retail online yang berasal dari United Kingdom mempunyai data penjualan berbagai produk selama periode satu tahun (November 2016 - Desember 2017). Toko tersebut menjual hadiah (gift) di platform online-nya.   

# Project Objective

Temukan pelanggan yang melakukan pembelian yang tinggi. Perusahaan ingin meluncurkan program loyalitas / loyality program kepada pelanggan yang mempunyai high-value setelah mengidentifikasi segmen.


---

# Import

## Library

In [ ]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 
from pandas.plotting import parallel_coordinates

from datetime import datetime
import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

 ## Dataset

In [ ]:
df = pd.read_csv('Ecommerce.csv', encoding='ISO-8859-1')
df.head()

 ## Dataset Content

1. Invoice No  
Purchase invoice no, unique for transaction

2. Stock Code  
Purchased item stock code, unique item code

3. Description  
Item description

4. Quantity  
No of items purchased

5. Invoice Date
Date of purchase

6. Unit Price  
Item unit price

7. Customer ID  
Customer unique reference

8. Country  
Country of location

---

# Preprocessing

 ## Tipe data  

 Membetulkan tipe data.

In [ ]:
df.info()

In [ ]:
df.drop('Unnamed: 8', axis=1, inplace=True)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df['CustomerID'] = df['CustomerID'].astype(str)

In [ ]:
df.dtypes

In [ ]:
df['CustomerID'] = df['CustomerID'].apply(lambda x: x.replace('.0',''))

 ## Menentukan tahun

In [ ]:
df['InvoiceYear'] = pd.to_datetime(df['InvoiceDate']).dt.year

df.head()

In [ ]:
df['InvoiceYear'].value_counts()

Karena data terbanyak terdapat pada tahun 2017, maka data tersebut yang akan dipakai

In [ ]:
df = df[df['InvoiceYear']==2017]

df['InvoiceYear'].value_counts()

 ## Missing Value

In [ ]:
df.isna().sum()

In [ ]:
df.drop('Description', axis=1, inplace=True)

In [ ]:
df[df['CustomerID']=='nan'].count()['CustomerID']

 ## Statistik Deskriptif

In [ ]:
df.describe()

Dari hasil statistik deskriptif menunjukan bahwa ada nilai yang tidak benar di kolom Quantity dan UnitPrice. Nilai tersebut tidak seharusnya kurang dari nol

In [ ]:
row, column = df.shape

In [ ]:
describe = pd.DataFrame({
    'Columns':['Quantity','UnitPrice'],
    'Summary':[len(df[df['Quantity']<1]),len(df[df['UnitPrice']<1])],
    'Percentage':[(len(df[df['Quantity']<1])/row)*100,(len(df[df['UnitPrice']<1])/row)*100]
})

describe

Walaupun jumlah data yang tidak valid pada kolom UnitPrice sangat banyak (21%), data tersebut tidak akan dipakai karena untuk menghindari kesalahan

In [ ]:
df = df.loc[(df['Quantity']>0) & (df['UnitPrice']>0) & (df['CustomerID']!='nan')]

In [ ]:
df.describe().round(3)

 ## Menambah kolom total sales

In [ ]:
df['TotalSales'] = df['Quantity']*df['UnitPrice']
df.head()

---

# Recency, Frequency dan Monetary   


Menggunakan data Recency, Frequency dan Monetary yang sudah terbukti untuk mempresentasikan aktifitas transaksi atau perilaku pelanggan di berbagai industri.  
* Recency  
Berapa lama setelah transaksi terakhir pelanggan
* Frequency  
Berapa kali transaksi yang dilakukan pelanggan
* Monetary  
Berapa yang yang sudah dibelanjakan oleh pelanggan


 ## Tanggal referensi  

 Membuat tanggal referensi untuk analisis dan segmentasi.

In [ ]:
ref_date = max(df['InvoiceDate']) + datetime.timedelta(days=1)

print('Tanggal referensi adalah:', ref_date.date())

## Aggregasi data per invoice

In [ ]:
df['Day'] = df['InvoiceDate'].dt.date

In [ ]:
df_inv = df.groupby(['Day','InvoiceNo','CustomerID'], as_index=False).agg({'TotalSales':'sum'})

df_inv

In [ ]:
df_inv.groupby('CustomerID', observed=True).count()

 ## Aggregate data untuk tiap customer

Step ini dilakukan untuk aggregasi tiap pelanggan agar mendapatkan nilai recency, frequency dan monetary.

In [ ]:
customer = df_inv.groupby(['CustomerID']).agg({
    'Day':lambda x: (ref_date.date()-x.max()).days,
    'InvoiceNo':'count',
    'TotalSales':'sum'
})

customer.rename(columns={
    'Day':'Recency',
    'InvoiceNo':'Frequency',
    'TotalSales':'Monetary'
}, inplace=True)

customer

---

# Exploratory Data Analysis

In [ ]:
customer.describe().round(2)

Dari hasil di atas, dapat diketahui bahwa:
* rata-rata pelanggan melakukan transaksi 85 hari yang lalu
* rata-rata pelanggan melakukan transaksi sebanyak 4 kali
* rata-rata pelanggan mengeluarkan uang sebesar 2332 pound sterling

## Distribusi Data

In [ ]:
plt.style.use('seaborn')

In [ ]:
sns.pairplot(customer, plot_kws={'alpha':0.8}, height=3)
plt.show()

Nilai Frequency dan Monetary berkumpul di sisi sebelah kiri yang artinya frekuensi pelanggan melakukan transaksi rendah dan pelanggan rata-rata mengeluarkan uang sedikit.

Karena terdapat skewness pada data, akan dilakukan data transformasi menggunakan log untuk membuat distribusi data mendekati normal agar hasil model yang didapatkan akan lebih optimal.

 ## Log Transform

In [ ]:
log_customer = np.log10(customer)

log_customer.head()

In [ ]:
sns.pairplot(log_customer, plot_kws={'alpha':0.8}, height=3)
plt.show()

 ## Scaling

Model Machine Learning yang akan digunakan untuk segmentasi adalah K-Means. Untuk mendapatkan hasil yang lebih optimal, perlu melakukan standarisasi data agar seluruh data memiliki scala yang sama. Dikarenakan algoritma K-Means akan menentukan cluster berdasarkan jarak antar titik ke titik yang lain.

In [ ]:
scaler = StandardScaler()

scaler.fit(log_customer)
customer_scaled = scaler.transform(log_customer)

customer_scaled = pd.DataFrame(customer_scaled, index=log_customer.index, columns=['Recency_scaled','Frequency_scaled','Monetary_scaled'])

customer_scaled.head()

In [ ]:
sns.kdeplot(customer_scaled['Recency_scaled'], shade=True)
sns.kdeplot(customer_scaled['Frequency_scaled'], shade=True)
sns.kdeplot(customer_scaled['Monetary_scaled'], shade=True)

---

# Machine Learning

Segmentasi akan menggunakank algoritma K-Means. K-Means merupakan salah satu teknik unsupervised Machine Learning.  

Salah satu nilai penting yang harus dimasukkan pada algoritma K-Means adalah jumlah cluster yang diinginkan. Oleh karena itu, perlu mencari berapa jumlah cluster yang optimal untuk data yang di punya. Salah satu cara untuk mendapatkan nilai optimal tersebut adalah menggunakan bantuan Silhouette Method.

## Silhouette Method

In [ ]:
silhouette_score_list = []
number_of_cluster = range(2,11)

for i in number_of_cluster:
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(customer_scaled)

    labels = kmeans.labels_
    silhouette_score_list.append(silhouette_score(customer_scaled, labels, metric='euclidean'))

In [ ]:
plt.figure(figsize=(10,8))
sns.lineplot(number_of_cluster, silhouette_score_list)
sns.scatterplot(number_of_cluster, silhouette_score_list)

plt.title('Silhouette Score')
plt.xlabel('Number of Cluster')
plt.ylabel('Average Within Sumsquare')
plt.show()

Silhouette Score terbaik terdapat pada cluster berjumlah 2.

## Fitting

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(customer_scaled)

customer_scaled['Cluster'] = kmeans.labels_
customer_scaled

In [ ]:
sns.pairplot(customer_scaled, plot_kws={'alpha':0.8,'size':0.5}, height=3, hue='Cluster')
plt.show()

In [ ]:
sns.set_style('dark')

df_plot = customer_scaled.groupby(['Cluster'], as_index=False).mean().round(2)

fig = plt.figure(figsize=(15,5))
title = fig.suptitle('Parallel Coordinates', fontsize=18)

pc = parallel_coordinates(df_plot, 'Cluster', color=['blue','red'])

Dari hasil parallel coordinates plot tersebut dapat diketahui bahwa urutan nilai yang dimiliki oleh tiap cluster dan bisa mengetahui mana cluster yang baik, yaitu cluster 0.

## Menggabungkan dengan data asli

In [ ]:
df_join = customer_scaled.merge(customer, how='inner', left_index=True, right_index=True)

df_result = df_join[['Recency','Frequency','Monetary','Cluster']]

In [ ]:
df_result.groupby('Cluster').mean()

## Nama Segmen Pelanggan  

Dari hasil segmentasi, didapatkan 2 segmen pelanggan:
* Cluster 0 -> Loyalist: pelanggan yang aktif dan sering bertransaksi
* Cluster 1 -> Loss: pelanggan yang lama tidak bertransaksi dengan rata-rata freqeuncy dan monetary yang rendah

In [ ]:
df_result['Segment'] = df_result['Cluster'].map({
    0:'Loyalist',
    1:'Loss'
})

df_result

In [ ]:
f, ax = plt.subplots(1,2, figsize=(15,10))

g = sns.barplot(x = df_result['Segment'].value_counts(),
                y = df_result['Segment'].value_counts().index,
                order = ['Loyalist','Loss'],
                color = 'lightblue',
                ax = ax[0])

g = sns.barplot(x = df_result['Segment'].value_counts(),
                y = df_result['Segment'].value_counts().index,
                color = 'lightblue',
                ax = ax[1])

ax[0].set(title = 'Number of Users in Each Clusters Sorted by Cluster Values',
          xlabel = '',
          xticks = ([]))

ax[1].set(title = 'Number of Users in Each Clusters Sorted by Count Users',
          xlabel = '',
          xticks = ([]))

sns.despine(right=True, bottom=True, top=True)

labels = df_result['Segment'].value_counts().index

x_1 = df_result['Segment'].value_counts(sort=False, normalize=True)*100
x_1 = x_1.reindex(['Loyalist','Loss'])
x_2 = df_result['Segment'].value_counts()

for n, i in enumerate(labels):
 
  ax[0].text(x_1[n]-0.003,
             n, 
             s = f'{round(x_1[n],2):,}%', 
             va = 'center',
             ha = 'left', 
             color = 'black',
             fontsize=20)
  ax[1].text(x_2[n]-0.1, 
             n, 
             s = f'{round(x_2[n],2):,}', 
             va = 'center', 
             ha = 'right', 
             color = 'black',
             fontsize=20)

## Menyimpan hasil

In [ ]:
df_result.to_csv('user_segment.csv')

---

# Kesimpulan

Terdapat 2 segmentasi dari hasil clustering, yaitu Loyalist dan Loss. Customer dengan segmentasi Loyalist adalah pelanggan yang aktif bertransaksi dengan rata-rata transaksi terakhir 26 hari yang lalu, dengan rata-rata frequency belanja sebanyak 7 kali, serta mengeluarkan uang rata-rata sebesar 4.172 pound sterling.

Namun, pada tahun 2017 kebanyakan customer bersegmentasi Loss sebesar 59.23% atau 2.499 customer (dari total 4.219). Yaitu pelanggan yang lama tidak bertransaksi dengan rata-rata frequency dan monetary yang rendah.